In [16]:
import re
import pandas as pd
import numpy as np

In [21]:

def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = []
  object_entity = []
  subject_type= []
  object_type= []
  subject_start= []
  object_start= []
  subject_end= []
  object_end= []
  

  for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
    # i = i[1:-1].split(',')[0].split(':')[1]
    sub_entity = eval(i)
    ob_entity = eval(j)
    sub_word= sub_entity['word']
    sub_type = sub_entity['type']
    # j = j[1:-1].split(',')[0].split(':')[1]
    ob_word = ob_entity['word']
    ob_type = ob_entity['type']

    ob_start = ob_entity['start_idx']
    sub_start = sub_entity['start_idx']

    ob_end = ob_entity['end_idx']
    sub_end = sub_entity['end_idx']


    subject_entity.append(sub_word)
    object_entity.append(ob_word)
    subject_type.append(sub_type)
    object_type.append(ob_type)
    subject_start.append(sub_start)
    object_start.append(ob_start)
    subject_end.append(sub_end)
    object_end.append(ob_end)
  
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],
                              'subject_type' : subject_type, 'object_type': object_type,'sub_start' : subject_start,'ob_start':object_start,'sub_end' :subject_end ,
                              'ob_end': object_end,'source':dataset['source']})
  out_dataset.id = out_dataset.index
  return out_dataset


  
def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)
  
  return dataset

### 1. 기본 베이스 모델

In [5]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + ' [SEP] ' + e02
    concat_entity.append(temp)
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)

['Runch Randa [SEP] 방탄소년단', '위키백과 [SEP] 위키미디어 재단', '임재범 [SEP] 송남영', '산청군 [SEP] 이재근', '유신정우회 [SEP] 한국국민당']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 2. 단어를 토큰으로 바꾸기


In [6]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + ' [SEP] ' + e02
    concat_entity.append(temp)
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['Runch Randa [SEP] 방탄소년단', '위키백과 [SEP] 위키미디어 재단', '임재범 [SEP] 송남영', '산청군 [SEP] 이재근', '유신정우회 [SEP] 한국국민당']
0          <PER> 멤버로 데뷔하기 전에 그는 《<PER>》라는 이름인 언더 래퍼였다.
1    <ORG>를 운영하고 있는 <ORG>은 매년 세계적인 컨퍼런스인 위키마니아 행사를 ...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, <PE...
3    경남 <ORG> <ORG>청에서 열린 협약식에는 최재호 무학그룹 회장과 <PER> ...
4    1980년 구 민주공화당 인사들과 <ORG> 인사들이 <ORG>을 창당하였고, 19...
Name: sentence, dtype: object


### 3. 인덱스 스타트 번호 넣기

In [7]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, s01, s02 in zip(dataset['subject_entity'], dataset['object_entity'], dataset['sub_start'], dataset['ob_start']):
    temp = ''
    temp = e01 + ' [SEP] '+ f'{s01}' + ' [SEP] ' + e02+ ' [SEP] '+ f'{s02}'
    concat_entity.append(temp)
  
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #                                                                 dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['Runch Randa [SEP] 22 [SEP] 방탄소년단 [SEP] 0', '위키백과 [SEP] 0 [SEP] 위키미디어 재단 [SEP] 14', '임재범 [SEP] 43 [SEP] 송남영 [SEP] 52', '산청군 [SEP] 7 [SEP] 이재근 [SEP] 36', '유신정우회 [SEP] 19 [SEP] 한국국민당 [SEP] 30']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 4. 스타트 번호, 끝번호 둘다 넣기

In [8]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, s01, s02, d01, d02 in zip(dataset['subject_entity'], dataset['object_entity'], dataset['sub_start'], dataset['ob_start'], dataset['sub_end'], dataset['ob_end']):
    temp = ''
    temp = e01 + ' [SEP] '+ f'{s01} {d01}' + ' [SEP] ' + e02+ ' [SEP] '+ f'{s02} {d02}'
    concat_entity.append(temp)
  
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #                                                                 dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['Runch Randa [SEP] 22 32 [SEP] 방탄소년단 [SEP] 0 4', '위키백과 [SEP] 0 3 [SEP] 위키미디어 재단 [SEP] 14 21', '임재범 [SEP] 43 45 [SEP] 송남영 [SEP] 52 54', '산청군 [SEP] 7 9 [SEP] 이재근 [SEP] 36 38', '유신정우회 [SEP] 19 23 [SEP] 한국국민당 [SEP] 30 34']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 5. object이랑 subject 각각 표시

In [9]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = '<ob> ' + e01 + ' [SEP] ' + '<sub> ' + e02
    concat_entity.append(temp)
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)

['<ob> Runch Randa [SEP] <sub> 방탄소년단', '<ob> 위키백과 [SEP] <sub> 위키미디어 재단', '<ob> 임재범 [SEP] <sub> 송남영', '<ob> 산청군 [SEP] <sub> 이재근', '<ob> 유신정우회 [SEP] <sub> 한국국민당']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 6. 타입별로 모델을 다르게 만드는것 

### 7. 한문장에 다 때려박기

In [10]:

def change_sentence(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f' <s> <{sub_type}> </s>',sentence)
  sentence = re.sub(rf'{ob_word}',f' <o> <{ob_type}> </o>',sentence)

  return sentence

def tokenized_dataset(dataset):
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_sentence(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  # print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_type.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

0    〈Something〉는  <o> <PER> </o>이 쓰고  <s> <ORG> </...
1    호남이 기반인 바른미래당· <o> <ORG> </o>· <s> <ORG> </s>이...
2    K리그2에서 성적 1위를 달리고 있는  <s> <ORG> </s>는 지난 26일  ...
3    균일가 생활용품점 (주) <s> <ORG> </s>(대표  <o> <PER> </o...
4     <o> <DAT> </o>년 프로 야구 드래프트 1순위로  <s> <ORG> </...
Name: sentence, dtype: object


### 8. 토큰을 object,subject 구분하기 - 실패

In [11]:

def change_word2(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f' <S. {sub_type}> ',sentence)
  sentence = re.sub(rf'{ob_word}',f' <O. {ob_type}> ',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
#   print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_type.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

0    〈Something〉는 <PER>이 쓰고 <ORG>가 1969년 앨범 《Abbey ...
1    호남이 기반인 바른미래당·<ORG>·<ORG>이 우여곡절 끝에 합당해 민생당(가칭)...
2    K리그2에서 성적 1위를 달리고 있는 <ORG>는 지난 26일 <ORG>으로부터 관...
3    균일가 생활용품점 (주)<ORG>(대표 <PER>)는 코로나19 바이러스로 어려움을...
4    <DAT>년 프로 야구 드래프트 1순위로 <ORG>에게 입단하면서 등번호는 8번으로...
Name: sentence, dtype: object


### 9. loss function focal로 바꿔보기 - 14번이랑 비교

In [ ]:
# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits

#         # calculate focal loss
#         alpha = [0.139, 0.179, 0.212, 0.221, 0.815, 1.891, 6.016, 8.694]
#         alpha = torch.tensor(alpha).to(self.args.device)
#         gamma = 2.0

#         ce_loss = torch.nn.functional.cross_entropy(logits, labels, reduction='none')
#         pt = torch.exp(-ce_loss)
#         focal_loss = alpha[labels] * (1-pt)**gamma * ce_loss

#         loss = torch.mean(focal_loss)

#         return (loss, outputs) if return_outputs else loss
    

### 10. 이름도 넣고 라벨도 넣는 방법 

In [12]:

def change_sentence2(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f' <s> {sub_word} <{sub_type}> </s> ',sentence)
  sentence = re.sub(rf'{ob_word}',f' <o> {ob_word} <{ob_type}> </o> ',sentence)

  return sentence

def tokenized_dataset(dataset):
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_sentence2(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  # print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_type.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

0    〈Something〉는  <o> 조지 해리슨 <PER> </o> 이 쓰고  <s> ...
1    호남이 기반인 바른미래당· <o> 대안신당 <ORG> </o> · <s> 민주평화당...
2    K리그2에서 성적 1위를 달리고 있는  <s> 광주FC <ORG> </s> 는 지난...
3    균일가 생활용품점 (주) <s> 아성다이소 <ORG> </s> (대표  <o> 박정...
4     <o> 1967 <DAT> </o> 년 프로 야구 드래프트 1순위로  <s> 요미...
Name: sentence, dtype: object


### 11. 2번째 문장에 10번 내용 넣기 
##### 기존 sentence 그대로 두고 2번째 문장에 태그와 타입 이름 모두 넣는다 .

In [13]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, t01, t02 in zip(dataset['subject_entity'], dataset['object_entity'],dataset['subject_type'] ,dataset['object_type']):
    temp = ''
    temp =  f'<s> {e01} <{t01}> </s>' + ' [SEP] ' + f'<o> {e02} <{t02}> </o> '
    concat_entity.append(temp)
  
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_type.csv")
tokenized_train = tokenized_dataset(train_dataset)

['<s> 비틀즈 <ORG> </s> [SEP] <o> 조지 해리슨 <PER> </o> ', '<s> 민주평화당 <ORG> </s> [SEP] <o> 대안신당 <ORG> </o> ', '<s> 광주FC <ORG> </s> [SEP] <o> 한국프로축구연맹 <ORG> </o> ', '<s> 아성다이소 <ORG> </s> [SEP] <o> 박정부 <PER> </o> ', '<s> 요미우리 자이언츠 <ORG> </s> [SEP] <o> 1967 <DAT> </o> ']
0    〈Something〉는 <PER>이 쓰고 <ORG>가 1969년 앨범 《Abbey ...
1    호남이 기반인 바른미래당·<ORG>·<ORG>이 우여곡절 끝에 합당해 민생당(가칭)...
2    K리그2에서 성적 1위를 달리고 있는 <ORG>는 지난 26일 <ORG>으로부터 관...
3    균일가 생활용품점 (주)<ORG>(대표 <PER>)는 코로나19 바이러스로 어려움을...
4    <DAT>년 프로 야구 드래프트 1순위로 <ORG>에게 입단하면서 등번호는 8번으로...
Name: sentence, dtype: object


### 12. 11번에 8번 합치기

In [14]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, t01, t02 in zip(dataset['subject_entity'], dataset['object_entity'],dataset['subject_type'] ,dataset['object_type']):
    temp = ''
    temp =  f'<s> {e01} <S. {t01}> </s>' + ' [SEP] ' + f'<o> {e02} <O. {t02}> </o> '
    concat_entity.append(temp)
    
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))

    
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)

['<s> Runch Randa <S. PER> </s> [SEP] <o> 방탄소년단 <O. PER> </o> ', '<s> 위키백과 <S. ORG> </s> [SEP] <o> 위키미디어 재단 <O. ORG> </o> ', '<s> 임재범 <S. PER> </s> [SEP] <o> 송남영 <O. PER> </o> ', '<s> 산청군 <S. ORG> </s> [SEP] <o> 이재근 <O. PER> </o> ', '<s> 유신정우회 <S. ORG> </s> [SEP] <o> 한국국민당 <O. ORG> </o> ']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 13. 11번에 기존 문장에 타입이랑 태그를 추가(기존 단어는 그대로 냅둠, 11번이랑 10번이랑 합친거와 같음)  - 현재 베스트

In [ ]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, t01, t02 in zip(dataset['subject_entity'], dataset['object_entity'],dataset['subject_type'] ,dataset['object_type']):
    temp = ''
    temp =  f'<s> {e01} <{t01}> </s>' + ' [SEP] ' + f'<o> {e02} <{t02}> </o> '
    concat_entity.append(temp)
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_sentence2(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))

  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_type.csv")
tokenized_train = tokenized_dataset(train_dataset)

### 14. 13번에서 <PER 말고 <S. PER> 과 같이 바꾸기 - 13번보다 높음 현재 기준

In [25]:

def change_sentence3(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f' <s> {sub_word} <S. {sub_type}> </s> ',sentence)
  sentence = re.sub(rf'{ob_word}',f' <o> {ob_word} <O. {ob_type}> </o> ',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, t01, t02 in zip(dataset['subject_entity'], dataset['object_entity'],dataset['subject_type'] ,dataset['object_type']):
    temp = ''
    temp =  f'<s> {e01} <S. {t01}> </s>' + ' [SEP] ' + f'<o> {e02} <O. {t02}> </o> '

    concat_entity.append(temp)
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_sentence3(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))

  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_type.csv")
tokenized_train = tokenized_dataset(train_dataset)

['<s> 비틀즈 <S. ORG> </s> [SEP] <o> 조지 해리슨 <O. PER> </o> ', '<s> 민주평화당 <S. ORG> </s> [SEP] <o> 대안신당 <O. ORG> </o> ', '<s> 광주FC <S. ORG> </s> [SEP] <o> 한국프로축구연맹 <O. ORG> </o> ', '<s> 아성다이소 <S. ORG> </s> [SEP] <o> 박정부 <O. PER> </o> ', '<s> 요미우리 자이언츠 <S. ORG> </s> [SEP] <o> 1967 <O. DAT> </o> ']
0    〈Something〉는  <o> 조지 해리슨 <O. PER> </o> 이 쓰고  <...
1    호남이 기반인 바른미래당· <o> 대안신당 <O. ORG> </o> · <s> 민주...
2    K리그2에서 성적 1위를 달리고 있는  <s> 광주FC <S. ORG> </s> 는...
3    균일가 생활용품점 (주) <s> 아성다이소 <S. ORG> </s> (대표  <o>...
4     <o> 1967 <O. DAT> </o> 년 프로 야구 드래프트 1순위로  <s>...
Name: sentence, dtype: object


### 15. 14번 뒤에 source 붙이기 - 조금 더 상승

In [26]:

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""

  concat_entity = []
  for e01, e02, t01, t02, source in zip(dataset['subject_entity'], dataset['object_entity'],dataset['subject_type'] ,dataset['object_type'], dataset['source']):
    temp = ''
    temp =  f'<s> {e01} <S. {t01}> </s>' + ' [SEP] ' + f'<o> {e02} <O. {t02}> </o> [SEP] <{source}>'

    concat_entity.append(temp)
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_sentence3(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))

  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_type.csv")
print(train_dataset.source.unique())
tokenized_train = tokenized_dataset(train_dataset)


['wikipedia' 'wikitree' 'policy_briefing']
['<s> 비틀즈 <S. ORG> </s> [SEP] <o> 조지 해리슨 <O. PER> </o> [SEP] <wikipedia>', '<s> 민주평화당 <S. ORG> </s> [SEP] <o> 대안신당 <O. ORG> </o> [SEP] <wikitree>', '<s> 광주FC <S. ORG> </s> [SEP] <o> 한국프로축구연맹 <O. ORG> </o> [SEP] <wikitree>', '<s> 아성다이소 <S. ORG> </s> [SEP] <o> 박정부 <O. PER> </o> [SEP] <wikitree>', '<s> 요미우리 자이언츠 <S. ORG> </s> [SEP] <o> 1967 <O. DAT> </o> [SEP] <wikipedia>']
0    〈Something〉는  <o> 조지 해리슨 <O. PER> </o> 이 쓰고  <...
1    호남이 기반인 바른미래당· <o> 대안신당 <O. ORG> </o> · <s> 민주...
2    K리그2에서 성적 1위를 달리고 있는  <s> 광주FC <S. ORG> </s> 는...
3    균일가 생활용품점 (주) <s> 아성다이소 <S. ORG> </s> (대표  <o>...
4     <o> 1967 <O. DAT> </o> 년 프로 야구 드래프트 1순위로  <s>...
Name: sentence, dtype: object
